In [1]:
import pandas as pd
import numpy as np
import regex as re
from tqdm import tqdm
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# pull scraped df from csv

raw_ticker_df = pd.read_csv(r'/Users/sam/Desktop/raw_ticker_df.csv')

# import industries/sectors from csv (via YF API pull)

ind_df = pd.read_csv(r'/Users/sam/Desktop/ticker_ind_df.csv')
sect_df = pd.read_csv(r'/Users/sam/Desktop/ticker_sect_df.csv')

# import growth from csv (via YF API pull)

growth_df = pd.read_csv(r'/Users/sam/Desktop/ticker_growth_df.csv')

In [4]:
# drop extra index column

raw_ticker_df = raw_ticker_df.drop(['Unnamed: 0'],axis=1)
sect_df = sect_df.drop(['Unnamed: 0'],axis=1)
ind_df = ind_df.drop(['Unnamed: 0'],axis=1)
growth_df = growth_df.drop(['Unnamed: 0'],axis=1)

In [5]:
df = pd.concat([raw_ticker_df,sect_df.sector,ind_df.industry,growth_df.growth],axis=1)

In [6]:
# asset book values are in thousands, converting to int and scaling up

df.mrq_assets = df.mrq_assets.str.replace(',','').astype(float) *1_000

In [7]:
# convert unclean data to manageable strings for processing, select columns

df.enterprise_value = df.enterprise_value.astype(str)
df.market_cap = df.market_cap.astype(str)
df.avg_vol = df.avg_vol.astype(str)
df.bso = df.bso.astype(str)
df.float_shares = df.float_shares.astype(str)
df.roa = df.roa.astype(str)
df.roe = df.roe.astype(str)
df.ltm_rev = df.ltm_rev.astype(str)
df.gross_profit = df.gross_profit.astype(str)
df.operating_margin = df.operating_margin.astype(str)
df.ltm_ebitda = df.ltm_ebitda.astype(str)

df.avg_vol = df.avg_vol.str.replace('3 month','three month')

In [8]:
# strip preceding text from select columns with string ints at the end

def strip_text(stats_text):
    clean_stats_list = []
    p = re.compile('[\-\d\.]+[\w%]')    
    for i in stats_text:
        try:
            result = p.findall(i)[0]
            clean_stats_list.append(result)
        except:
            result = '0'
            clean_stats_list.append(result)
    return clean_stats_list

In [9]:
# strip preceding text of columns

df.avg_vol = strip_text(df.avg_vol)
df.float_shares = strip_text(df.float_shares)
df.roa = strip_text(df.roa)
df.roe = strip_text(df.roe)
df.ltm_rev = strip_text(df.ltm_rev)
df.gross_profit = strip_text(df.gross_profit)
df.operating_margin = strip_text(df.operating_margin)
df.ltm_ebitda = strip_text(df.ltm_ebitda)

In [10]:
# cleans string values ending in characters 'B', 'M', 'k', else to zero, converts to float

def clean_trade_stats(stats_list):
    clean_stats_list = []
    for x in stats_list:
        x = str(x)
        if x[-1]=='T':
            x = x.replace('T','')
            x = float(x)
            x*=1_000_000_000_000
        elif x[-1]=='B':
            x = x.replace('B','')
            x = float(x)
            x*=1_000_000_000
        elif x[-1]=='M':
            x = x.replace('M','')
            x = float(x)
            x*=1_000_000
        elif x[-1]=='k':
            x = x.replace('k','')
            x = float(x)
            x*=1_000
        elif x[-1]=='%':
            x = x.replace('%','')
            x = float(x)
            x/=100
        else:
            x=0
            x = float(x)
        clean_stats_list.append(x)
    return clean_stats_list

In [11]:
# apply clean function to relevant columns

df.enterprise_value = clean_trade_stats(df.enterprise_value)
df.market_cap = clean_trade_stats(df.market_cap)
df.avg_vol = clean_trade_stats(df.avg_vol)
df.bso = clean_trade_stats(df.bso)
df.float_shares = clean_trade_stats(df.float_shares)
df.roa = clean_trade_stats(df.roa)
df.roe = clean_trade_stats(df.roe)
df.ltm_rev = clean_trade_stats(df.ltm_rev)
df.gross_profit = clean_trade_stats(df.gross_profit)
df.operating_margin = clean_trade_stats(df.operating_margin)
df.ltm_ebitda = clean_trade_stats(df.ltm_ebitda)

In [12]:
# confirm dtypes

df.dtypes

equity                  object
publishing_analysts      int64
enterprise_value       float64
market_cap             float64
avg_vol                float64
bso                    float64
float_shares           float64
roa                    float64
roe                    float64
ltm_rev                float64
gross_profit           float64
operating_margin       float64
ltm_ebitda             float64
mrq_assets             float64
sector                  object
industry                object
growth                 float64
dtype: object

In [13]:
# some industries (eg banking) do not have EVs... using market cap as a proxy for ratios

df.enterprise_value[df.enterprise_value==0] = df.market_cap

In [14]:
df.enterprise_value[df.enterprise_value==0]

125    0.0
368    0.0
369    0.0
740    0.0
924    0.0
Name: enterprise_value, dtype: float64

In [15]:
# dropping select tickers with no EVs or MCs
# these are obscure stocks that shouldn't impact the broader distribution meaningfully

i = df[((df.equity == 'BPYU') | 
        (df.equity == 'FWONA') | 
        (df.equity == 'FWONK') |
        (df.equity == 'QRTEA') | 
        (df.equity == 'VIACA'))].index
df.drop(i,inplace=True)
df.reset_index(drop=True)

,equity,publishing_analysts,enterprise_value,market_cap,avg_vol,bso,float_shares,roa,roe,ltm_rev,gross_profit,operating_margin,ltm_ebitda,mrq_assets,sector,industry,growth
0,A,17,4.807000e+10,4.633000e+10,31670000.0,2040000.0,3.024800e+08,0.0782,0.1931,5.820000e+09,2.840000e+09,0.2135,-1.550000e+09,9.627000e+09,Healthcare,Diagnostics & Research,0.232
1,AA,10,9.570000e+09,7.360000e+09,37680000.0,9640000.0,1.861400e+08,0.0464,0.1054,1.046000e+10,1.360000e+09,0.0985,1.700000e+09,1.486000e+10,Basic Materials,Aluminum,0.319
2,AAL,21,4.405000e+10,1.371000e+10,331860000.0,41800000.0,6.423700e+08,-0.0900,0.0000,1.869000e+10,-5.540000e+09,-0.5278,-7.650000e+09,6.200800e+10,Industrials,Airlines,3.610
3,AAP,26,1.646000e+10,1.385000e+10,3770690.0,519060.0,6.465000e+07,0.0518,0.1805,1.074000e+10,4.480000e+09,0.0923,1.240000e+09,1.183964e+10,Consumer Cyclical,Specialty Retail,0.234
4,AAPL,40,2.350000e+12,2.410000e+12,384690000.0,88210000.0,1.667000e+10,0.1690,1.0340,3.471600e+11,1.049600e+11,0.2732,1.109400e+11,3.238880e+11,Technology,Consumer Electronics,0.364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,ZION,20,8.570000e+09,8.570000e+09,31320000.0,1880000.0,1.598200e+08,0.0139,0.1454,3.190000e+09,2.400000e+09,0.4688,0.000000e+00,8.147900e+10,Financial Services,Banks—Regional,0.894
998,ZM,23,1.100500e+11,1.146300e+11,33140000.0,3960000.0,2.135700e+08,0.1476,0.3426,3.280000e+09,1.830000e+09,0.2865,9.736600e+08,5.297993e+09,Technology,Software—Application,1.914
999,ZNGA,13,1.119000e+10,1.112000e+10,313860000.0,13740000.0,1.030000e+09,-0.0006,-0.1453,2.250000e+09,1.160000e+09,-0.0019,1.754000e+08,6.206500e+09,Communication Services,Electronic Gaming & Multimedia,0.523
1000,ZS,27,3.196000e+10,3.247000e+10,31880000.0,1250000.0,1.058600e+08,-0.0819,-0.5470,6.019100e+08,3.355400e+08,-0.3047,-1.596900e+08,1.833458e+09,Technology,Software—Infrastructure,0.596


In [16]:
# create ratios from fundamentals

df['gross_margin'] = df.gross_profit / df.ltm_rev
df['ebitda_margin'] = df.ltm_ebitda / df.ltm_rev
df['ev_s'] = df.enterprise_value / df.ltm_rev
df['ev_ebitda'] = df.enterprise_value / df.ltm_ebitda

# replace inf and -inf values with zero to be imputed later with sector averages, temp value of zero

df.replace([np.inf,-np.inf],np.nan,inplace=True)
df.fillna(0,inplace=True)

In [17]:
# get median multiples excl nans and 0 values

med_ev_s = (df[df.ev_s!=np.nan].groupby('industry')
    .ev_s
    .agg('median')
    .rename('ind_med_ev_s')
    .reset_index())

med_ev_ebitda = (df[(df.ev_ebitda!=np.nan) & (df.ev_ebitda>0.0)].groupby('industry')
    .ev_ebitda
    .agg('median')
    .rename('ind_med_ev_ebitda')
    .reset_index())

med_gm = (df[(df.gross_margin<1) & (df.gross_margin>-.5) & (df.gross_margin!=0.0)].groupby('industry')
    .gross_margin
    .agg('median')
    .rename('ind_median_gm')
    .reset_index())

med_opm = (df[(df.operating_margin<.7) & (df.operating_margin>-.7) & (df.operating_margin!=0.0)].groupby('industry')
    .operating_margin
    .agg('median')
    .rename('ind_median_opm')
    .reset_index())

med_ebitda_margin = (df[(df.operating_margin<.75) & (df.operating_margin>-.7) & (df.ebitda_margin!=0.0)].groupby('industry')
    .ebitda_margin
    .agg('median')
    .rename('ind_median_ebitda_margin')
    .reset_index())

In [18]:
# merge industry median metrics

df = df.merge(med_ev_s,how='left',left_on='industry',right_on='industry')
df = df.merge(med_ev_ebitda,how='left',left_on='industry',right_on='industry')
df = df.merge(med_gm,how='left',left_on='industry',right_on='industry')
df = df.merge(med_opm,how='left',left_on='industry',right_on='industry')
df = df.merge(med_ebitda_margin,how='left',left_on='industry',right_on='industry')

In [19]:
# check for signficant margin outliers

print(df.gross_margin[(df.gross_margin>1.0) | (df.gross_margin<-.5)])
print(df.operating_margin[(df.operating_margin>.7) | (df.operating_margin<-.7)])
print(df.ebitda_margin[(df.ebitda_margin>.75) | (df.ebitda_margin<-.7)])

88     46764.189089
108        2.059867
155        5.914894
188        1.058696
356     6285.794105
586        1.020564
607      -11.435095
622        1.059132
742       -2.121796
872        1.222717
880       -1.363636
983       -0.720648
Name: gross_margin, dtype: float64
18    -1.3758
29     0.9560
46    -1.4091
64    -0.7421
209   -1.0061
232   -2.7327
259   -1.2742
301    6.5735
372   -1.0682
397   -1.5904
426   -1.1359
439    0.7209
473   -1.1646
490    0.7118
522   -1.4471
542   -0.9047
543   -2.2823
622   -3.1818
625    0.9725
700   -1.7837
707   -1.0102
708    9.6383
740   -0.9458
772   -0.9626
795   -0.9929
802    0.7858
807   -0.9171
824   -1.0254
867    0.7208
872   -0.8085
920    0.8671
983   -1.5879
Name: operating_margin, dtype: float64
18     -1.296782
46     -1.339493
88     -4.022271
155   -27.021277
232    -2.613669
259    -1.153865
276   -22.879881
356    -1.801928
372    -1.008493
377     0.869919
397    -0.979920
408     0.879518
522    -1.422028
542    -0.826733


In [20]:
# rectify margin outliers 

df.gross_margin[(df.gross_margin>1.0) | (df.gross_margin<-.5) | (df.gross_margin==0)] = df.ind_median_gm
df.operating_margin[(df.operating_margin>.7) | (df.operating_margin<-.7) | (df.operating_margin==0)] = df.ind_median_opm
df.ebitda_margin[(df.ebitda_margin>.75) | (df.ebitda_margin<-.7) | (df.ebitda_margin==0)] = df.ind_median_ebitda_margin

In [21]:
# dropping a handful of travel services and REITs; i'm okay dropping 
# these because REITs aren't widely covered anyways and travel cos had an exceptionally bad year w/ COVID

i = df[((df.equity == 'CCL') | 
        (df.equity == 'NCLH') | 
        (df.equity == 'RCL') |
        (df.equity == 'SRC') | 
        (df.equity == 'STOR') | 
        (df.equity == 'VER') | 
        (df.equity == 'VICI') | 
        (df.equity == 'WPC'))].index
df.drop(i,inplace=True)
df.reset_index(drop=True)

,equity,publishing_analysts,enterprise_value,market_cap,avg_vol,bso,float_shares,roa,roe,ltm_rev,...,growth,gross_margin,ebitda_margin,ev_s,ev_ebitda,ind_med_ev_s,ind_med_ev_ebitda,ind_median_gm,ind_median_opm,ind_median_ebitda_margin
0,A,17,4.807000e+10,4.633000e+10,31670000.0,2040000.0,3.024800e+08,0.0782,0.1931,5.820000e+09,...,0.232,0.487973,-0.266323,8.259450,-31.012903,8.414156,23.438643,0.501363,0.25980,0.294756
1,AA,10,9.570000e+09,7.360000e+09,37680000.0,9640000.0,1.861400e+08,0.0464,0.1054,1.046000e+10,...,0.319,0.130019,0.162524,0.914914,5.629412,0.914914,5.629412,0.130019,0.09850,0.162524
2,AAL,21,4.405000e+10,1.371000e+10,331860000.0,41800000.0,6.423700e+08,-0.0900,0.0000,1.869000e+10,...,3.610,-0.296415,-0.409310,2.356875,-5.758170,2.494174,NaN,-0.181631,-0.45670,-0.363190
3,AAP,26,1.646000e+10,1.385000e+10,3770690.0,519060.0,6.465000e+07,0.0518,0.1805,1.074000e+10,...,0.234,0.417132,0.115456,1.532588,13.274194,1.957912,14.340489,0.336705,0.11400,0.153847
4,AAPL,40,2.350000e+12,2.410000e+12,384690000.0,88210000.0,1.667000e+10,0.1690,1.0340,3.471600e+11,...,0.364,0.302339,0.319564,6.769213,21.182621,6.769213,21.182621,0.302339,0.27320,0.319564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,ZION,20,8.570000e+09,8.570000e+09,31320000.0,1880000.0,1.598200e+08,0.0139,0.1454,3.190000e+09,...,0.894,0.752351,NaN,2.686520,0.000000,3.794040,NaN,0.819853,0.43925,NaN
990,ZM,23,1.100500e+11,1.146300e+11,33140000.0,3960000.0,2.135700e+08,0.1476,0.3426,3.280000e+09,...,1.914,0.557927,0.296848,33.551829,113.027135,17.838875,68.982809,0.674397,0.05400,0.128449
991,ZNGA,13,1.119000e+10,1.112000e+10,313860000.0,13740000.0,1.030000e+09,-0.0006,-0.1453,2.250000e+09,...,0.523,0.515556,0.077956,4.973333,63.797035,5.751320,21.401573,0.674817,0.19240,0.221891
992,ZS,27,3.196000e+10,3.247000e+10,31880000.0,1250000.0,1.058600e+08,-0.0819,-0.5470,6.019100e+08,...,0.596,0.557459,-0.265305,53.097639,-200.137767,9.444236,28.163742,0.681548,0.05180,0.106339


In [22]:
# check again for signficant outliers

print(df.gross_margin[(df.gross_margin>1.0) | (df.gross_margin<-.5)])
print(df.operating_margin[(df.operating_margin>.7) | (df.operating_margin<-.7)])
print(df.ebitda_margin[(df.ebitda_margin>.75) | (df.ebitda_margin<-.7)])

Series([], Name: gross_margin, dtype: float64)
Series([], Name: operating_margin, dtype: float64)
Series([], Name: ebitda_margin, dtype: float64)


In [23]:
# clean up minor nan values from ratio converion by filling with column mean

df.operating_margin.fillna(df.operating_margin.mean(),inplace=True)
df.ebitda_margin.fillna(df.ebitda_margin.mean(),inplace=True)
df.ind_med_ev_ebitda.fillna(df.ind_med_ev_ebitda.mean(),inplace=True)
df.ind_median_opm.fillna(df.ind_median_opm.mean(),inplace=True)
df.ind_median_ebitda_margin.fillna(df.ind_median_ebitda_margin.mean(),inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 994 entries, 0 to 1001
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   equity                    994 non-null    object 
 1   publishing_analysts       994 non-null    int64  
 2   enterprise_value          994 non-null    float64
 3   market_cap                994 non-null    float64
 4   avg_vol                   994 non-null    float64
 5   bso                       994 non-null    float64
 6   float_shares              994 non-null    float64
 7   roa                       994 non-null    float64
 8   roe                       994 non-null    float64
 9   ltm_rev                   994 non-null    float64
 10  gross_profit              994 non-null    float64
 11  operating_margin          994 non-null    float64
 12  ltm_ebitda                994 non-null    float64
 13  mrq_assets                994 non-null    float64
 14  sector   

In [25]:
df.avg_vol[df.avg_vol==0] = np.median(df.avg_vol)
df.bso[df.bso==0] = np.median(df.bso)
df.float_shares[df.float_shares==0] = np.median(df.avg_vol)

In [26]:
# set minimum floor for listed volume, create proxy for daily trading volume

df.float_shares[df.float_shares<1_250_000] = 1_250_000
df['daily_vol'] = .01 * df.float_shares

In [27]:
# eliminate growth outliers, some software will have >150% growth

df.growth[df.growth >3.0] = 3.0

In [28]:
# back into revenue figure for 5 observations with no ltm_rev (due to scrape flaws)

df.ltm_rev[df.ltm_rev==0] = df.enterprise_value / df.ind_med_ev_s

In [29]:
# create dummies for sector

sector_dummies = pd.get_dummies(df['sector'],drop_first=True)
sector_dummies.columns = ['comm_services','cons_cycl','cons_def',
                          'energy','fin_services','healthcare',
                          'industrials','niche','real_estate','tech',
                          'utilities']
df = pd.concat([df,sector_dummies],axis=1)

In [30]:
clean_df = df[['equity','sector','industry','publishing_analysts',
            'enterprise_value','market_cap','daily_vol',
            'mrq_assets','ltm_rev','gross_margin',
            'operating_margin','ebitda_margin','roa',
            'roe','growth','comm_services','cons_cycl','cons_def',
            'energy','fin_services','healthcare',
            'industrials','niche','real_estate','tech',
            'utilities']].reset_index(drop=True)

In [31]:
clean_df.to_csv(r'/Users/sam/Desktop/clean_ticker_df.csv')